In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import multivariate_normal
from scipy.integrate import quad
from scipy.integrate import nquad
from scipy.integrate import dblquad
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import ParameterGrid
from fancyimpute import MICE 

print ('done')

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


done


In [2]:
# Function: artificial datasets generation with/without missing values

# Input:
# total_samples: total number of samples in the dataset
# bias between class 0 and class 1. bias_1 + bias_2 = 1 
# mean_1 and mean_2: mean array for each class. 1*3 array.
# cov_1 and cov_2: covariance array for each class. 3*3 array.
# frac_missing_list: missing data fraction of each feature in each class. np 2*3 array.

def dataset_generation(total_samples, bias_1, mean_1, mean_2, cov_1, cov_2, frac_missing_list):
    # Generate 2 classes
    samples_c1 = int(total_samples * bias_1)
    samples_c2 = int(total_samples * (1 - bias_1))
    label_c1 = np.zeros(samples_c1, dtype=int)
    label_c2 = np.ones(samples_c2, dtype=int)
    x_1, y_1, z_1 = np.random.multivariate_normal(mean_1, cov_1, samples_c1).T
    x_2, y_2, z_2 = np.random.multivariate_normal(mean_2, cov_2, samples_c2).T
    
    class_1 = pd.DataFrame({'Feature_1':x_1, 'Feature_2':y_1, 'Feature_3':z_1,'Class_label':label_c1})
    class_2 = pd.DataFrame({'Feature_1':x_2, 'Feature_2':y_2, 'Feature_3':z_2,'Class_label':label_c2})
    print('Class_1 and Class_2 info:', class_1.info(), class_2.info())
    
    # Put in missing values
    update = class_1.sample(frac=frac_missing_list.item(0,0))
    update.Feature_1 = 'N/A'
    class_1.update(update)
    
    update = class_1.sample(frac=frac_missing_list.item(1,0))
    update.Feature_2 = 'N/A'
    class_1.update(update)
    
    update = class_1.sample(frac=frac_missing_list.item(2,0))
    update.Feature_3 = 'N/A'
    class_1.update(update)
    
    update = class_2.sample(frac=frac_missing_list.item(0,1))
    update.Feature_1 = 'N/A'
    class_2.update(update)
    
    update = class_2.sample(frac=frac_missing_list.item(1,1))
    update.Feature_2 = 'N/A'
    class_2.update(update)
    
    update = class_2.sample(frac=frac_missing_list.item(2,1))
    update.Feature_3 = 'N/A'
    class_2.update(update)
        
    # combine and shuffle the dataset
    complete = pd.concat([class_1,class_2],axis=0) # combine
    complete.Class_label = complete.Class_label.astype(int)
    complete.replace({'N/A': np.nan}, inplace=True)
    complete = complete.sample(frac=1).reset_index(drop=True) # shuffle
    
    # save dataset
    complete.to_csv('/Users/yy10/Documents/missing data paper/data/artificial_data_3features.csv',index=False)
   
    return complete
    

In [3]:
# Inputs

total_samples = 100

bias_1 = 0.5
bias_2 = 1e0-bias_1

mean_1 = [0, 0, 0] 
mean_2 = [5e0, 5e0, 5e0]

cov_1 = [[1e0, 0, 0], [0, 1e0, 0], [0, 0, 1e0]]
cov_2 = [[1e0, 0, 0], [0, 1e0, 0], [0, 0, 1e0]]

frac_missing_list = np.array([[0.8,0.7],[0.8,0.5],[0.6,0.5]])

# dataset generation

df = dataset_generation(total_samples, bias_1, mean_1, mean_2, cov_1, cov_2, frac_missing_list)
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
Class_label    50 non-null int64
Feature_1      50 non-null float64
Feature_2      50 non-null float64
Feature_3      50 non-null float64
dtypes: float64(3), int64(1)
memory usage: 1.6 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
Class_label    50 non-null int64
Feature_1      50 non-null float64
Feature_2      50 non-null float64
Feature_3      50 non-null float64
dtypes: float64(3), int64(1)
memory usage: 1.6 KB
Class_1 and Class_2 info: None None


,Class_label,Feature_1,Feature_2,Feature_3
0,0,NaN,NaN,-0.360980
1,1,NaN,NaN,NaN
2,1,6.422475,6.496080,NaN
3,0,NaN,NaN,NaN
4,1,NaN,5.673988,4.383160
5,0,NaN,NaN,NaN
6,1,NaN,NaN,4.538691
7,1,NaN,NaN,NaN
8,1,4.101909,NaN,NaN
9,0,NaN,NaN,0.317198


In [4]:
# calculate two 2-d Gaussian distribution overlaping integral: baseline accuracy

# Function that chooses the larger value of the two Gaussian pdf for a given set of x,y
rv_1 = multivariate_normal(mean_1, cov_1)
rv_2 = multivariate_normal(mean_2, cov_2)
gauss_overlap = lambda x,y,z,bias_1,bias_2: np.maximum.reduce([rv_1.pdf([x,y,z])*bias_1,\
                                                               rv_2.pdf([x,y,z])*bias_2])

# Analytical solution of the integral of the two gaussian's overlaping area
# first number is the solution, the second number is the error
#dblquad(gauss_overlap, -np.inf, np.inf, lambda x: -np.inf, lambda x: np.inf,args=(bias_1,bias_2))
print('The baseline accuracy of the generated dataset is:',
      nquad(gauss_overlap,[[-np.inf,np.inf],[-np.inf,np.inf],[-np.inf,np.inf]],args=(bias_1,bias_2)))

# Plotting contour of gauss_overlap
#x, y = np.mgrid[-5:10:.01, -5:10:.01]
#pos = np.empty(x.shape + (2,))
#pos[:, :, 0] = x; pos[:, :, 1] = y
#gauss_overlap = lambda x: np.maximum.reduce([rv_1.pdf(x)*bias_1,\
                                             #rv_2.pdf(x)*bias_2])
#plt.contourf(x, y, gauss_overlap(pos))
#plt.title('Contour plot of max(Gaussian_1, Gaussian_2) ')
#plt.xlabel('Feature 1')
#plt.ylabel('Feature 2')
#plt.show()

# Plot the 2 classes without missing values
#plt.plot(x_1, y_1, z_1, 'o', color='k')
#plt.plot(x_2, y_2, z_2, 'o', color='r')
#plt.axis('equal')
#plt.show()
    

The baseline accuracy of the generated dataset is: (0.6674972497207666, 1.4899689561554486e-08)
